In [ ]:
# To cleare computed data before git commit, do:
# - Kernel > Restart & Clear Output
# - Edit > Cut Cell Attachments
# - Save
# gimme more width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1500px !important; }</style>"))

In [ ]:
import plotly.express as px
import pandas
from opensimplex import OpenSimplex
import datetime
import random

o = OpenSimplex() #seed=1)

density = 13
k = 40

ts = datetime.datetime.now()

d = {}
for u in range(10*density):
    for v in range(10*density):
        for w in range(10*density):
            x = u / 100
            y = v / 100
            z = w / 100
            val = o.noise3d(x/density*k, y/density*k, z/density*k)
            pval = abs(val)
            if pval > .45: # and random.random() > pval:
                d.setdefault('x', []).append(x)
                d.setdefault('y', []).append(y)
                d.setdefault('z', []).append(z)
                d.setdefault('v', []).append(val if val > 0 else 0.5 + val)
df = pandas.DataFrame(d)
print(datetime.datetime.now() - ts)
print(len(df))

In [ ]:
import plotly.express as px

fig1 = px.scatter_3d(df, x='x', y='y', z='z', color='v', opacity=0.05, height=800)
fig1.show()  # cannot set marker size, so no control on density

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig2 = go.Figure()
fig2.add_trace(
    go.Scatter3d(
        x=df['x'], y=df['y'], z=df['z'],
        mode='markers',
        marker_colorscale='Plasma',
        marker=dict(
            color=df['v'],
            size=4,
            opacity=0.03,
            sizemode='diameter',
            line=dict(width=0),
            colorbar=dict(thickness=20)
        )
    )
)
fig2.update_layout(height=800)

fig2.show()

In [ ]:
import numpy
import math
from skimage import measure
import plotly.graph_objects as go

################################################################################
# Volume function

density = 13
k = 40
def scalar_func(x, y, z):
    val = o.noise3d(x/density*k, y/density*k, z/density*k)
    return abs(val) - 0.45  # volume defined by low noise

func = numpy.vectorize(scalar_func)

################################################################################
# Isosurface

def compute_volume(span, resolution, f):
    x, y, z = span * numpy.mgrid[0:1:resolution,
                                 0:1:resolution,
                                 0:1:resolution]
    volume = f(x, y, z)
    vertices, faces, normals, vals = measure.marching_cubes_lewiner(
        volume, 0,
        spacing=(0.1, 0.1, 0.1),
        allow_degenerate=True)
    return vertices, faces

resolution = 99j
span = 1
vertices, faces = compute_volume(span, resolution, func)

################################################################################
# 3D plot

x,y,z = zip(*vertices)
i,j,k = zip(*faces)

# https://plotly.com/python/3d-mesh/

fig = go.Figure(data=[
    go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k)
])

fig.update_layout(height=800)
fig.show()